In [3]:
# --- Clean Helmholtz / pickup calculator (SymPy) ---

from sympy import symbols, Eq, pi, sqrt, solve, acos, atan, N

# ---------- Inputs (edit here) ----------
# Geometry (transmit Helmholtz pair)
N_tx      = 15            # turns per coil (identical coils)
R_in      = 2.0           # *inches* (coil radius; 4" diameter -> 2")
ell_in    = 1.0           # *inches* (winding pack length along axis)
k_couple  = 0.30          # coupling factor between the two coils (0..1), ~0.2–0.4 typical for Helmholtz spacing

# Series capacitor (for series-resonant drive)
C = 0.66e-6               # F  (ZVC board value)

# Receive coil ("firefly")
N_rx      = 10            # turns
d_rx_m    = 1.0*0.0254    # m  (1" diameter)
A_rx      = pi*(d_rx_m/2)**2

# Target induced voltage (RMS) and frequency choice
V_rx_rms  = 2.5           # V (desired RMS on the receive coil)
use_resonance = True      # True: drive at series resonance; False: specify freq_Hz below
freq_Hz   = 30000.0       # used only when use_resonance=False

# Copper losses / series resistance in each Tx coil (set what you measure/estimate)
R1 = 0.20                 # ohm (coil 1 ESR at the operating frequency)
R2 = 0.20                 # ohm (coil 2 ESR at the operating frequency)

# ---------- Constants ----------
mu0 = 4*pi*1e-7
R_m = R_in * 0.0254       # m (coil radius)
# Helmholtz separation is usually equal to R; we assume that geometry for the field formula.

# ---------- Wheeler helpers ----------
def wheeler_single_layer_uH(r_in, ell_in, N):
    """Wheeler single-layer solenoid (r, l in inches) -> L in microhenries."""
    return (r_in**2 * N**2) / (9*r_in + 10*ell_in)

def wheeler_multilayer_uH(r_avg_in, ell_in, N, d_in):
    """Wheeler multilayer (approx): L(µH) ≈ 0.8 r_avg^2 N^2 / (6 r_avg + 9 ℓ + 10 d)."""
    return 0.8 * (r_avg_in**2) * (N**2) / (6*r_avg_in + 9*ell_in + 10*d_in)

# ---------- Inductances of the Tx coils ----------
L1 = wheeler_single_layer_uH(R_in, ell_in, N_tx) * 1e-6   # H
L2 = L1

# Include mutual coupling (important for Helmholtz pair):
M  = k_couple * sqrt(L1*L2)
L_series = L1 + L2 + 2*M      # effective series inductance seen by the driver

# ---------- Frequency / resonance ----------
if use_resonance:
    f0 = 1/(2*pi*sqrt(L_series*C))
    omega = 2*pi*f0
else:
    omega = 2*pi*freq_Hz
    f0 = omega/(2*pi)

# ---------- Field required for the target receive voltage ----------
# V_rx,rms = (N_rx * A_rx * omega * B0) / sqrt(2)   (pickup aligned to B)
B0 = V_rx_rms*sqrt(2) / (N_rx * A_rx * omega)      # Tesla (field amplitude)

# ---------- Current required in Helmholtz coils ----------
# B0 = (4/5)^(3/2) * mu0 * N_tx * I0 / R_m
I0_peak = B0 * R_m / (mu0 * N_tx) * (5/4)**(3/2)   # peak
I_rms   = I0_peak / sqrt(2)                        # RMS current through the series pair

# ---------- Driver voltage (series R-L-C) ----------
# Z = R_tot + j(omega*L_series - 1/(omega*C))
R_tot   = R1 + R2
X_tot   = omega*L_series - 1/(omega*C)
V_drv_rms = I_rms * sqrt(R_tot**2 + X_tot**2)      # RMS source voltage required
phase_deg  = atan(X_tot/R_tot) * 180/pi            # current lag (+) or lead (-)

# ---------- Print summary ----------
print(f"Each Tx coil (Wheeler single-layer): L1=L2 = {N(L1,6)} H")
print(f"Coupling k = {k_couple:.2f}  ->  M = {N(M,6)} H")
print(f"Effective series inductance L_series = {N(L_series,6)} H")

print(f"\nSeries resonance f0 = {N(f0,6)} Hz")
print(f"Operating angular freq ω = {N(omega,6)} rad/s")

print(f"\nTarget receive V_rms = {V_rx_rms} V with N_rx={N_rx}, A_rx={N(A_rx,6)} m^2")
print(f"Required field amplitude B0 = {N(B0,6)} T")

print(f"\nTx current: I_peak = {N(I0_peak,6)} A,   I_rms = {N(I_rms,6)} A")

print(f"\nSeries R: R1+R2 = {R_tot} Ω,  Reactance X = {N(X_tot,6)} Ω")
print(f"Required driver V_rms = {N(V_drv_rms,6)} V   (current phase = {N(phase_deg,4)}°)")


Each Tx coil (Wheeler single-layer): L1=L2 = 0.0000321429 H
Coupling k = 0.30  ->  M = 0.00000964286 H
Effective series inductance L_series = 0.0000835714 H

Series resonance f0 = 21429.9 Hz
Operating angular freq ω = 134648 rad/s

Target receive V_rms = 2.50000000000000 V with N_rx=10, A_rx=0.000506707 m^2
Required field amplitude B0 = 0.00518201 T

Tx current: I_peak = 19.5176 A,   I_rms = 13.8010 A

Series R: R1+R2 = 0.400000000000000 Ω,  Reactance X = 1.24345E-14 Ω
Required driver V_rms = 5.52041 V   (current phase = 1.781E-12°)
